## 서브쿼리란?
- 다른 SELECT문에 삽입된 SELECT문 입니다.
- 테이블 자체 데이터에 종속된 조건을 사용하여 테이블에서 행을 선택할 때 유용
- 서브쿼리가 먼저 실행되고, 그 결과를 사용하여 메인 쿼리에서 사용
- 사용지침
    - 괄호()로 묶어야함
    - 비교조건의 오른쪽에 서브쿼리를 넣음
    - ORDER BY절을 사용하지 않음
    - 단일행 연산자(>,<,=,<>등)와 복수행 연산자(IN,ANY,ALL)를 사용
    - SELECT절에 사용하는 경우 컬럼1개, 행1개만 추출하는 형태의 서브쿼리만 가능 (그 외는 여러 행이 나오는 서브쿼리 사용가능)

---

## WHERE절 서브쿼리
- WHERE절에 사용된 서브쿼리 실행순서는 서브쿼리를 먼저 수행하고 그 결과값을 이용해 메인 쿼리 조건을 비교

서동명 선수와 같은 팀 선수들 이름, 팀id, 포지션 정보를 조회

<pre>
SELECT player_name, team_id, position
FROM player
WHERE team_id = ( SELECT team_id
                  FROM player
                  WHERE player_name = '서동명' );
</pre>

전체 선수들 평균키 보다 키가 큰 선수들만 선수id, 이름, 키, 팀id 정보를 조회

<pre>
SELECT player_id, player_name, height ,team_id
FROM player
WHERE height > ( SELECT avg(height) 
                 FROM player );
</pre>

포지션은 서동명 선수와 같고 키는 디디 선수보다 큰 선수들만 이름, 포지션 ,키 조회

<pre>
SELECT player_name, position, height
FROM player
WHERE position = ( SELECT position 
                   FROM player
                   WHERE player_name='서동명' )
AND height > ( SELECT height
               FROM player
               WHERE player_name='디디' );
</pre>

---

## 다중행 서브쿼리
- 서브쿼리 결과가 여러행을 반환하는 경우 다중 행 서브쿼리

    - IN : 서브쿼리 결과 목록에 있는 임의의 값과 같은지 비교
    
    
    - ANY : 같은 반환되는 각각의 값과 개별 비교해서 조건을 만족하는지 확인
        - '< ANY' : 최대값보다 적음을 의미
        - '> ANY' : 최소값보다 큼을 의미
        - '= ANY' : IN과 동일하게 처리
        
        
    - ALL : 서브쿼리에 의해 반환되는 모든 값과 비교
        - '< ALL' : 최소값보다 적음을 의미
        - '> ALL' : 최대값보다 큼을 의미

player 테이블 K02팀에 소속된 모든 선수들보다 키가 큰 선수들의 이름, 키를 조회

<pre>
SELECT player_name, height
FROM player
WHERE height > ALL( SELECT height
                    FROM player
                    WHERE team_id = 'K02'); 
</pre>
<br>

ALL 키워드대신 최대키값과 비교한 결과를 확인

<pre>
SELECT player_name, height
FROM player
WHERE height > ( SELECT MAX(height)
                 FROM player
                 WHERE team_id = 'K02'); 
</pre>

player 테이블 K02팀에 소속된 최소 한명의 선수보다 키가 큰 선수들이 있으면 이름, 키를 조회

<pre>
SELECT player_name, height
FROM player
WHERE height > ANY ( SELECT height
                     FROM player
                     WHERE team_id = 'K02'); 
</pre>
<br>


ANY 키워드대신 최소키값과 비교한 결과를 확인
<pre>
SELECT player_name, height
FROM player
WHERE height > ( SELECT MIN(height)
                 FROM player
                 WHERE team_id = 'K02');
</pre>                 
             

---

## FROM절 서브쿼리
- 서브쿼리가 FROM절에 명시되면 새로운 데이터 소스를 생성
- FROM절에 기술한 서브쿼리는 마치 뷰를 생성하고 뷰를 SELECT문에서 부르는 것과 개념상 비슷
- 데이터 소스처럼 사용되는 서브쿼리는 저장되지 않음
- SELECT문 일부로서 일시적으로만 존재
- 인라인 뷰라고도 함

team, player이용 팀명, 연고지명, 해당팀 선수들 평균키를 조회

<pre>
SELECT t.team_name, t.region_name, avg(p.height) 
FROM player p 
JOIN team t 
ON p.team_id = t.team_id 
GROUP BY t.team_name, t.region_name;
</pre>
- player테이블 모든 자료를 team테이블과 조인한 다음 팀명 , 연고지로  GROUP BY 작업을 수행
<br>
---


서브쿼리를 FROM절에 사용하여 조회
<pre>
SELECT t.team_name, t.region_name, p.avg_h
FROM    
(
  	SELECT team_id, avg(height) avg_h
    FROM player
    GROUP BY team_id 
) p 
JOIN team t 
ON p.team_id = t.team_id;
</pre>
- player테이블 자료를 팀id기준 GROUP BY작업을 먼저 수행후 team테이블과 조인 작업을 수행

---

## 스칼라 서브쿼리란?
- 하나의 행, 하나의 컬럼값만 반환하는 서브쿼리를 스칼라 서브쿼리라고 함
- 스칼라 서브 쿼리의 값은 서브 쿼리의 SELECT 목록에 있는 항목 값
- 서브쿼리가 0개의 행을 반환하면 스칼라 서브쿼리의 값은 NULL이 됨
- SELECT, ORDER BY절에는 스칼라 서브쿼리 형태인 문장만 사용 가능

player테이블에서 선수이름, 팀id, 키, player 테이블 전체 평균키를 조회

<pre>
SELECT player_name, team_id, height,( SELECT avg(height) FROM  player) avg_h
FROM player;
</pre>

### 스칼라 서브쿼리 사용시 주의점
- 열1개, 행1개만 추출하는 서브쿼리를 사용
- 여러 개의 컬럼을 조회하거나 여러 행을 추출하는 경우 에러가 발생

---

## 상관 서브 쿼리란?
- 메인 쿼리의 한 행에 대해서 서브쿼리가 한번씩 실행
- 테이블에서 행을 먼저 읽어서 각 행의 값을 관련된 데이터와 비교하는 방법
- 서브 쿼리가 다른 결과를 반환해야하는 경우에 사용
- 각 행의 값에 따라 응답이 달라지는 질의에 응답할때 상호연관 서브쿼리를 사용
- 서브쿼리에서 메인 쿼리의 컬럼명을 사용 할 수 있으나, 메인 쿼리에서는 서브쿼리의 컬럼명을 사용할 수 없음

### 상호연관 서브쿼리 실행
1. 후보 행을 가져옴 (메인쿼리에서 인출)
2. 후보 행의 값을 사용하여 서브쿼리를 실행
3. 서브 쿼리의 결과값을 사용하여 후보 행의 조건을 확인
4. 후보 행이 남지 않을 때까지 반복

player 선수중 K07팀 평균키보다 키가 큰 선수들의 이름, 키, 팀id를 조회

<pre>
SELECT p.player_name, p.height, p.team_id
FROM player p 
WHERE p.height > ( SELECT avg(height) 
                   FROM player
                   WHERE team_id = 'K07');
</pre>
<br>

player 선수중 본인이 소속된 팀 평균키보다 키가 큰 선수들의 이름, 키, 팀id를 조회
<pre>
SELECT p.player_name, p.height, p.team_id
FROM player p 
WHERE p.height > ( SELECT avg(height) 
                   FROM player
                   WHERE team_id = p.team_id);
</pre>
<br>

포지션 GK인 선수들의 선수이름, 키, 팀id, K05팀 평균키를 조회
<pre>
SELECT p.player_name, p.height, p.team_id,
( SELECT avg(height) FROM player WHERE team_id ='K05' )  avg_h
FROM player p
WHERE p.position = 'GK';
</pre>

---

## EXISTS 사용하기
- EXISTS 연산자는 외부 질의에서 검색된 값이 내부 질의의 결과에 존재하는지 여부를 검사하기 위한 상호 연관 서브쿼리에서 자주 사용
- 서브 쿼리가 한 행 이상 반환되면 TRUE를 반환하고, 해당 값이 없으면 FALSE를 반환

team테이블 자료중 키가 190보다 큰 선수가 한명이라도 존재하면 해당 팀id, 팀이름, 연고지명을 조회

<pre>
SELECT t.team_id, t.team_name, t.region_name
FROM team t  
WHERE EXISTS ( SELECT 1
               FROM player
               WHERE height > 190 
			   AND team_id = t.team_id);
</pre>
<br>

구장 중 특정 팀의 전용구장으로 사용되는 구장 정보만 구장이름, 좌석수, 홈팀id를 조회
<pre>
SELECT s.stadium_name, s.seat_count, s.hometeam_id
FROM stadium s
WHERE EXISTS ( SELECT 1
               FROM team
               WHERE team_id = s.hometeam_id);
</pre>

---

## NOT EXISTS 사용하기
- NOT EXISTS는 외부 질의에서 검색된 값이 내부 질의결과에 존재하지 않으면 TRUE, 존재하면 FALSE를 반환

team테이블 자료중 키가 190보다 큰 선수가 없는 팀들만 팀id, 팀이름, 연고지명을 조회
<pre>
SELECT t.team_id, t.team_name, t.region_name
FROM team t  
WHERE NOT EXISTS ( SELECT 1
                   FROM player
                   WHERE height > 190
				   AND team_id = t.team_id);
</pre>
<br>

구장 중 특정 팀의 전용구장으로 사용되지 않는 구장 정보만 구장이름을 조회
<pre>
SELECT s.stadium_name
FROM stadium s
WHERE NOT EXISTS ( SELECT 1
                   FROM  team
                   WHERE team_id = s.hometeam_id);
</pre>